In [2]:
%matplotlib notebook
import pandas as pd
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt

df = pd.read_csv("Output/venue_information_partial.csv")
df.head(50)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue ID,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State,Likes,Rating
0,0,0,0,Wakefield,40.894705,-73.847201,The Upper Room,4f57ef61e4b063a4ab675858,Music Venue,40.892567,-73.846406,NaN,New York,NaN,NaN
1,1,1,1,Wakefield,40.894705,-73.847201,241st Liquor Store,4f349b15e4b0e4d04e99a428,Other Nightlife,40.902771,-73.849898,Bronx,NY,NaN,NaN
2,2,2,2,Wakefield,40.894705,-73.847201,Dyme Life Radio,4da63d645da3e8486b80546f,Music Venue,40.894541,-73.843266,Bronx,NY,NaN,NaN
3,3,3,3,Wakefield,40.894705,-73.847201,Par-City,4d23aea014f2b1f7740f68be,Music Venue,40.890211,-73.847002,Bronx,NY,NaN,NaN
4,4,4,4,Wakefield,40.894705,-73.847201,Tavern,4f32cf0919836c91c7fd025d,Bar,40.895898,-73.855731,Bronx,NY,NaN,NaN
5,5,5,5,Wakefield,40.894705,-73.847201,Inn At the Border,511b2022e4b08d6c8cf67ff2,Other Nightlife,40.898248,-73.838641,Mount Vernon,NY,NaN,NaN
6,6,6,6,Wakefield,40.894705,-73.847201,226th,4fc16c3ae4b008d93ac9fd5a,Bar,40.887470,-73.854510,Bronx,NY,NaN,NaN
7,7,7,7,Wakefield,40.894705,-73.847201,MY Studio,4e9cb64c49017c00c840e759,Music Venue,40.901926,-73.853504,NaN,New York,NaN,NaN
8,8,8,8,Wakefield,40.894705,-73.847201,Major League Music Group,4ee0764f0aafbb3dbab41e92,Music Venue,40.891367,-73.850088,NaN,New York,NaN,NaN
9,9,9,9,Wakefield,40.894705,-73.847201,M.O.S. Cigars,57fd59a2498e721d3b3e14bf,Smoke Shop,40.890628,-73.848478,Bronx,NY,NaN,NaN
